### BW33 : Yield from을 사용하여 여러 제너레이터를 합성하라
- yield from : 부모 제너레이터에 전달하기 전에 내포된 제너레이터가 모든 값을 내보냄
    

In [8]:
import timeit

In [51]:
# yield 로 제너레이터 생성
def move(period, speed):
    for _ in range(period):
        yield speed
        
def pause(delay):
    for _ in range(delay):
        yield 0

In [52]:
# yield 로 제너레이터 생성
# for 문과 yield 가 너무 반복적이라서 가독성이 떨어짐, 잡음 늘어남
def animate():
    for delta in move(4,5.0):
        yield delta
    for delta in pause(3):
        yield delta
    for delta in move(2,3.0):
        yield delta

In [53]:
# 확인 코드
def render(delta):
    print(f'Delta: {delta:.1f}')

def run(func):
    for delta in func():
        render(delta)

t1 = timeit.timeit()
run(animate)
t2 = timeit.timeit()
print('걸린 시간 : ',t2-t1)

Delta: 5.0
Delta: 5.0
Delta: 5.0
Delta: 5.0
Delta: 0.0
Delta: 0.0
Delta: 0.0
Delta: 3.0
Delta: 3.0
걸린 시간 :  0.0022093679999670712


In [45]:
# 위의 식은 제너레이터를 결국 두 번 만들어서 하나의 동작을 하는데, 여기선 yield from으로 한번의 제너레티어를 만들어서 모든 동작 바로 구현
def animate_composed():
    yield from move(4,5.0)
    yield from pause(3)
    yield from move(2,3.0)

t3 = timeit.timeit()
run(animate_composed)
t4 = timeit.timeit()
print('걸린 시간 : ',t4-t3)

Delta: 5.0
Delta: 5.0
Delta: 5.0
Delta: 5.0
Delta: 0.0
Delta: 0.0
Delta: 0.0
Delta: 3.0
Delta: 3.0
걸린 시간 :  -0.0008100570003080065


### 성능 확인    
**timeit 사용 timeit.timeit(stmt, setup, timer, number, globals)**.   
- stmt : 실행시간을 확인하고 싶은 코드(default : pass)    
- setup : setup 디테일. stmt 전에 실행할 것(default : pass)
- timer : 이미 default값이 정해져 있음.무시해도 됨
- number : stmt를 실행할 횟수(default : 1000000)
- globals : 코드를 실행할 이름 공간 지정(globals(), locals(), namespace)

In [50]:
# 책에서 나온 성능 확인 방법
import timeit

def child():
    for i in range(1_000_000):
        yield i

def slow():
    for i in child():
        yield i
    
def fast():
    yield from child()
    
baseline = timeit.timeit(stmt='for _ in slow(): pass', globals=globals(),number=50)
print(f'수동 내포 : {baseline:.2f}s')

comparison = timeit.timeit(stmt='for _ in fast(): pass', globals=globals(),number=50)
print(f'합성 사용 : {comparison:.2f}s')

수동 내포 : 3.91s
합성 사용 : 3.52s


In [38]:
from functools import reduce
print(reduce(lambda x, y: y+x , 'abcde'))
# reduce(lambda x, y: x + y, [0, 1, 2, 3, 4])

edcba


In [31]:
for x in (['abcde']):
    print(x)

abcde


### BW36 : itertools 를 사용하라


In [58]:
import itertools
it1,it2,it3 = itertools.tee(['한','둘'],3)
print(list(it1))
print(list(it2))
print(list(it3))

['한', '둘']
['한', '둘']
['한', '둘']


In [59]:
# bw31
def normalize(numbers):
    total = sum(numbers)
    result = []
    for value in numbers:
        percent = 100 * value / total
        result.append(percent)
    return result

visits = [15, 35, 80]
percentages = normalize(visits)
print(percentages)

[11.538461538461538, 26.923076923076923, 61.53846153846154]


In [89]:
# f = open('my_numbers.txt','w')
# f.write('15 ')
# f.write('35 ')
# f.write('80\n')
# f.close()

# f = open('my_numbers.txt','r')
# print(f.readline())
# for line in f:
#     a = f.readlines()
#     print(a)

In [92]:
def read_visits(data_path):
    with open(data_path) as f:
        for line in f:
            yield int(line)

it = read_visits('my_numbers.txt')
percentages = normalize(it)
print(percentages)

[]


15 35 80

